In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
# from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.image import resize

ModuleNotFoundError: No module named 'ucimlrepo'

In [ ]:
# Fetch the Wine dataset (ID 109 on UCI)
wine = fetch_ucirepo(id=109)

# Extract features and targets as pandas DataFrames
X = wine.data.features
y = wine.data.targets

# Print metadata and variable information for reference
print(wine.metadata)
print(wine.variables)

# Split dataset: 60% training, 20% validation, 20% testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Display sizes of each set to confirm the split
print(f"Training set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")


In [ ]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Reshape scaled data
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_val_reshaped = X_val_scaled.reshape(X_val_scaled.shape[0], X_val_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train['class'])
y_val_encoded = label_encoder.transform(y_val['class'])
y_test_encoded = label_encoder.transform(y_test['class'])

# Get number of classes
K = len(np.unique(y_train_encoded))

# Build the improved CNN model
model = models.Sequential([
    layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], 1), kernel_regularizer=l2(0.01)),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=128, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)),
    layers.MaxPooling1D(pool_size=1),
    layers.Conv1D(filters=256, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)),
    layers.GlobalAveragePooling1D(),
    layers.Dense(units=256, activation='relu', kernel_regularizer=l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(units=K, activation='softmax')
])

# Compile the model with a lower learning rate
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

In [ ]:
history = model.fit(X_train_reshaped, y_train_encoded,
                    validation_data=(X_val_reshaped, y_val_encoded),
                    epochs=40, batch_size=32, verbose=1)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_encoded, verbose=0)

# Predict on test data
y_pred = model.predict(X_test_reshaped)
y_pred_classes = y_pred.argmax(axis=1)

# Calculate confusion matrix, precision, and recall
conf_matrix = confusion_matrix(y_test_encoded, y_pred_classes)
precision = precision_score(y_test_encoded, y_pred_classes, average='weighted')
recall = recall_score(y_test_encoded, y_pred_classes, average='weighted')

# Print results
print(f"Train Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()